<a href="https://colab.research.google.com/github/zororo2/pals/blob/main/Monthly_exposure_21_06_2024.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Check if notebook is running in Google Colab
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

In [2]:
if IN_COLAB:
    # Use the token from Github to clone the PRECISE repository with read/write access
    from IPython.display import clear_output; user="kudzaishezororo"; token=input();
    !git clone https://github.com/zororo2/pals.git

    clear_output()
    # Install relevant packages
    !pip install geehydro cartopy
    !pip install rioxarray
    !pip install mapclassify
    clear_output()

In [3]:
import os
import folium, cartopy, mapclassify
import geehydro
import geopandas as gpd
import numpy as np, pandas as pd
import cartopy.crs as ccrs
import ee
import rioxarray as rio, xarray as xr
import geemap
from matplotlib import pyplot as plt
import matplotlib.dates as mdates

In [4]:
# Authenticate and initialise Earth Engine API
try:
    ee.Initialize(project="ee-monthlyexposures")
except Exception as e:
    ee.Authenticate()
    ee.Initialize(project="ee-monthlyexposures")

In [6]:
def generateImageCollection(exposure, country, dataset, facilities=False):

    path='shapefiles/precise_villages.gpkg'
    path=os.path.join(r'/content/precisehealthgeo', path) if IN_COLAB else path
    start='2018-11-01'
    end='2023-12-31'
    ### image processing functions ###
    ##################################

    # load the shapefile to geodataframe
    if facilities:
        gdf=gpd.read_file('/content/pals/shapefiles1/precise_villages6.gpkg', layer='{}_health_facilities'.format(country))
        #gdf=gpd.read_file(path, layer='{}_health_facilities'.format(country))

        # gdf=gpd.read_file('./shapefiles/precise_villages.gpkg', layer=country)
        # gdf=gpd.GeoDataFrame(gdf.drop(columns='geometry'), geometry=gdf.geometry.centroid, crs=gdf.crs) # get village centroids
    else:
        gdf=gpd.read_file('pals/shapefiles1/precise_villages6.gpkg', layer=country)
        #gdf=gpd.read_file(path, layer=country)
    # convert gdf to ee feature collection
    roi=(ee.FeatureCollection(geemap.gdf_to_ee(gdf))\
         .set('country', country))

    # generate image collection for the study period and apply functions
    if dataset=='landsat':
        # masks out clouds
        def mask_clouds(image):
            # landsat quality assesment band
            qaBand=image.select('QA_PIXEL')
            # bits 5 and 3 are cloud and cloud shadow respectively
            cloudBitMask=1<<5
            cloudShadowBitMask=1<<3
            # both bits should be equal to zero indicating clear consitions
            mask=(qaBand.bitwiseAnd(cloudBitMask).eq(0)\
                .And(qaBand.bitwiseAnd(cloudShadowBitMask).eq(0)))
            # apply the mask to the optical and thermal bands
            return (image.updateMask(mask)\
                    .select('SR_B.*', 'ST_B10')
                    .copyProperties(image, ["system:time_start"]))

        # applies landsat scaling factors
        def scale_image(image):
            opticalBands=image.select('SR_B.*').multiply(0.0000275).add(-0.2)
            thermalBand=image.select('ST_B10').multiply(0.00341802).add(149.0).subtract(273.15).rename('lst')
            return (image.addBands(opticalBands, None, True)\
                    .addBands(thermalBand, None, True))

        # computes Normalised Difference Vegetation Index
        def calculate_ndvi(image):
            ndvi=image.normalizedDifference(['SR_B5', 'SR_B4']).rename('ndvi')
            return image.addBands(ndvi)

        collection=(ee.ImageCollection('LANDSAT/LC08/C02/T1_L2')\
                    .filterBounds(roi)
                    .filterDate(start, end) #('2018-11-01', '2018-12-31') this period illustrates clouds for mozambique
                    .map(mask_clouds)
                    .map(scale_image)
                    .map(calculate_ndvi))

    elif dataset=='landsat-7':
        # masks out clouds
        def mask_clouds(image):
            # landsat quality assesment band
            qaBand=image.select('QA_PIXEL')
            # bits 5 and 3 are cloud and cloud shadow respectively
            cloudBitMask=1<<5
            cloudShadowBitMask=1<<3
            # both bits should be equal to zero indicating clear consitions
            mask=(qaBand.bitwiseAnd(cloudBitMask).eq(0)\
                .And(qaBand.bitwiseAnd(cloudShadowBitMask).eq(0)))
            # apply the mask to the optical and thermal bands
            return (image.updateMask(mask)\
                    .select('SR_B.*', 'ST_B6')
                    .copyProperties(image, ["system:time_start"]))

        # applies landsat scaling factors
        def scale_image(image):
            opticalBands=image.select('SR_B.*').multiply(0.0000275).add(-0.2)
            thermalBand=image.select('ST_B6').multiply(0.00341802).add(149.0).subtract(273.15).rename('lst')
            return (image.addBands(opticalBands, None, True)\
                    .addBands(thermalBand, None, True))

        # computes Normalised Difference Vegetation Index
        def calculate_ndvi(image):
            ndvi=image.normalizedDifference(['SR_B4', 'SR_B3']).rename('ndvi')
            return image.addBands(ndvi)

        collection=(ee.ImageCollection('LANDSAT/LE07/C02/T1_L2')\
                    .filterBounds(roi)
                    .filterDate(start, end)
                    .map(mask_clouds)
                    .map(scale_image)
                    .map(calculate_ndvi))

    elif dataset=='modis':
        # applies modis scaling factors
        def scale_image(image):
            if exposure=='ndvi':
                return (image.select('NDVI').multiply(0.0001).rename('ndvi')\
                        .copyProperties(image, ["system:time_start"]))
            else: # include 'LST_Night_1km' later
                return (image.select('LST_Day_1km').multiply(0.02).subtract(273.15).rename('lst')\
                        .copyProperties(image, ["system:time_start"]))
        # changing this to modis daily
        collection=ee.ImageCollection("MODIS/061/MOD13Q1") if exposure=='ndvi' else ee.ImageCollection('MODIS/061/MOD11A1')
        collection=(collection.filterBounds(roi)\
                    .filterDate(start, end)
                    .map(scale_image))

    elif dataset=='era5':
        # applies era5 scaling factors
        def scale_image(image): #change ear5 to skin temperature
            return (image.select('skin_temperature').subtract(273.15).rename('lst')\
                    .copyProperties(image, ["system:time_start"]))

        collection=(ee.ImageCollection("ECMWF/ERA5_LAND/DAILY_AGGR")\
            # ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR')\
                    .filterBounds(roi)
                    .filterDate(start, end)
                    .map(scale_image))

          #Computes the Precipitation of the ROI
    elif dataset=='persiann-cdr':

        def scale_image(image):
            return (image.select('precipitation').multiply(2.5).rename('lst')\
                    .copyProperties(image, ["system:time_start"]))


        collection=(ee.ImageCollection('NOAA/PERSIANN-CDR')\
                    .filterBounds(roi)
                    .filterDate(start,end)
                    .map(scale_image))


    elif dataset=='merra2':
            #calculate the wet bulb global temperature

            def scale_image(image):
                return (image.select('T2MWET').rename('lst')\
                        .copyProperties(image, ["system:time_start"]))

            collection=(ee.ImageCollection("NASA/GSFC/MERRA/slv/2")\
                        .filterBounds(roi)
                        .filterDate(start,end)
                        .map(scale_image))


    elif dataset=='trmm':
            #calculate the precipitation

            def scale_image(image):
                return (image.select('precipitation').rename('lst')\
                        .copyProperties(image, ["system:time_start"]))

            collection=(ee.ImageCollection("TRMM/3B43V7")\
                        .filterBounds(roi)
                        .filterDate(start,end)
                        .map(scale_image))

    return collection.select(exposure), roi

In [14]:
# we can visualise our image collection on a map just to check
def drawCollection(collection, exposure, country):
    # map centres
    getCenter=dict(gambia=[13.443, -15.864], mozambique=[-25.1914, 32.7539], kenya=[-3.9995, 39.3609])
    # color palette
    viz={'ndvi': dict(min=-0.2, max=1, palette='8bc4f9, c9995c, c7d270, 8add60, 097210'), 'lst': dict(min=0, max=60, palette='6495ed, 32cd32, fdda0d, 8b4000, ff0000')}
    # Use folium to visualize the image collection
    map=folium.Map(location=getCenter[country], zoom_start=8)
    map.addLayer(collection[0], viz[exposure])
    map.addLayer(collection[1])
    return map

In [7]:
def generateMonthlyTimeSeries(input_collection, exposure, country, dataset):
    # get date range of image collection
    start=ee.Date(input_collection[0].aggregate_min('system:time_start'))
    end=ee.Date(input_collection[0].aggregate_max('system:time_start'))
    n_months=end.difference(start, 'months').add(1)
    months=ee.List.sequence(0, n_months.int())
    # generate unique dates for analysis period
    dates=months.map(lambda i: start.advance(i, 'month'))

    # Groups images by month and computes mean
    def monthly_agg(date, collection):
        start=ee.Date(date)
        end=ee.Date(date).advance(1, 'month')
        collection=collection.filterDate(start, end).mean() #pixel-wise mean for entire collection
        return (collection.set('system:time_start', start.millis())\
                .set('count', collection.bandNames().length())) #this helps us identify months without images

    # generate monthly mean image collection
    mean_monthly=ee.ImageCollection.fromImages(dates.map(lambda i: monthly_agg(i, input_collection[0]))\
                                                 .filter(ee.Filter.gt('count', 0)))  #retain only non-null images

    crs_list=dict(gambia='EPSG:32628', mozambique='EPSG:32736', kenya='EPSG:32737')
    # Computes mean value for each neighborhood
    def reduceMean(image):
        features=image.reduceRegions(
            reducer=ee.Reducer.mean(),
            collection=input_collection[1],
            scale=100,
            crs=crs_list[country])
        return features.map(lambda f: f.set('exposure_month', image.date().format()))

    # Computes mean value for each health facility
    def extractPoint(image):
        features=image.sampleRegions(
            collection=input_collection[1],
            scale=100,
            geometries=True,
            projection=crs_list[country])
        return features.map(lambda f: f.set('exposure_month', image.date().format()))

    if input_collection[1].geometry().type().getInfo()[5:]=='Polygon':
        # generate monthly mean by village for image collection
        exposures=mean_monthly.map(reduceMean)
        # export to dataframe and set new index
        exposures=(geemap.ee_to_df(exposures.flatten())\
                   .rename(columns={'mean': exposure}))
        for col in exposures.columns:
            if col in ['name', 'locality', 'district', 'admin_post']:
                exposures=exposures.drop(columns=col)
        # change exposure month datetime format
        exposures['exposure_month']=exposures['exposure_month'].apply(lambda x: pd.to_datetime(x).strftime('%Y_%m'))
        return (exposures.set_index(['neighborhood_code', 'exposure_month'])\
                .sort_index())
    else:
        # extract monthly values at facility points
        exposures=mean_monthly.map(extractPoint)
        # export to dataframe and set new index
        exposures=geemap.ee_to_df(exposures.flatten())
        for col in exposures.columns:
            if col in ['name', 'nature', 'Name', 'locality', 'district', 'admin_post']:
                exposures=exposures.drop(columns=col)
        # change exposure month datetime format
        exposures['exposure_month']=exposures['exposure_month'].apply(lambda x: pd.to_datetime(x).strftime('%Y_%m'))
        return (exposures.set_index(['facility_code', 'exposure_month']) #(exposures.set_index(['neighborhood_code', 'exposure_month']) ##
                .sort_index())

In [8]:
def aggregateTimeSeries(input_collection, exposure, country, dataset):
    county_list=dict(gambia=['Central Baddibu', 'Upper Baddibu'], mozambique=['Manhica'], kenya=['Kilifi', 'Kwale'])
    # utility for filtering by county
    county_filter=[ee.Filter.eq('ADM2_NAME', county) for county in county_list[country.lower()]]
    # get admin boundaries for country
    counties=(ee.FeatureCollection("FAO/GAUL/2015/level2")\
             .filter(ee.Filter.Or(*county_filter)))

    crs_list=dict(gambia='EPSG:32628', mozambique='EPSG:32736', kenya='EPSG:32737')

    # get date range of image collection
    start=ee.Date(input_collection[0].aggregate_min('system:time_start'))
    end=ee.Date(input_collection[0].aggregate_max('system:time_start'))
    n_months=end.difference(start, 'months').add(1)
    months=ee.List.sequence(0, n_months.int())
    # generate unique dates for analysis period
    dates=months.map(lambda i: start.advance(i, 'month'))

    # Groups images by month and computes mean
    def monthly_agg(date, collection):
        start=ee.Date(date)
        end=ee.Date(date).advance(1, 'month')
        collection=collection.filterDate(start, end).mean() #pixel-wise mean for entire collection
        return (collection.set('system:time_start', start.millis())\
                .set('count', collection.bandNames().length())) #this helps us identify months without images

    # generate monthly mean image collection
    mean_monthly=ee.ImageCollection.fromImages(dates.map(lambda i: monthly_agg(i, input_collection[0]))\
                                                 .filter(ee.Filter.gt('count', 0)))  #retain only non-null images

    # Computes mean value for entire image
    def reduceMean(image):
        mean=image.reduceRegion(
            reducer=ee.Reducer.mean(),
            geometry=counties,
            scale=30,
            crs=crs_list[country]
        ).get(exposure)
        return image.set(exposure, mean).set('exposure_month', image.date().format())

    # generate overall monthly mean for roi
    exposures=(mean_monthly.map(reduceMean)\
            .reduceColumns(ee.Reducer.toList(2), ['exposure_month', exposure]).values().get(0))

    # export to dataframe and set new index
    exposures=pd.DataFrame(exposures.getInfo(), columns=['exposure_month', exposure])
    # change exposure month datetime format
    exposures['exposure_month']=exposures['exposure_month'].apply(lambda x: pd.to_datetime(x).strftime('%Y_%m'))

    return exposures.set_index('exposure_month')

In [ ]:
def cimatologyMaps(input_collection, exposure, country, dataset):
    county_list=dict(gambia=['Central Baddibu', 'Upper Baddibu'], mozambique=['Manhica'], kenya=['Kilifi', 'Kwale'])
    # utility for filtering by county
    county_filter=[ee.Filter.eq('ADM2_NAME', county) for county in county_list[country.lower()]]
    # get admin boundaries for country
    counties=(ee.FeatureCollection("FAO/GAUL/2015/level2")\
             .filter(ee.Filter.Or(*county_filter)))

    crs_list=dict(gambia='EPSG:32628', mozambique='EPSG:32736', kenya='EPSG:32737')

    # get month range of a year
    months=ee.List.sequence(1, 12)

    def monthly_agg(m):
        return (input_collection[0].filter(ee.Filter.calendarRange(m, m, 'month'))\
                .mean()
                .set('month', m))

    # group by month and calculate monthly mean image collection
    mean_monthly=(ee.ImageCollection.fromImages(months.map(monthly_agg))\
                  .map(lambda i: i.clip(counties)))

    # export image to xarray
    lst_raster=geemap.ee_to_xarray(mean_monthly, crs='EPSG:4326', scale=30*0.001/111, geometry=counties.geometry())
    # specify nodata value
    lst_raster['lst']=(lst_raster['lst'].rio.write_nodata(np.nan)
                # specify spatial dimensions
                .rio.set_spatial_dims('lon', 'lat'))
    return lst_raster

In [9]:
def monthly_exposures(exposure, country, dataset, facilities=False, draw_map=False):
    images=generateImageCollection(exposure, country, dataset, facilities)
    return drawCollection(images, exposure, country) if draw_map else generateMonthlyTimeSeries(images, exposure, country, dataset)

In [10]:
def agg_monthly_exposures(exposure, country, dataset, facilities=False, draw_map=False):
    images=generateImageCollection(exposure, country, dataset, facilities)
    return drawCollection(images, exposure, country) if draw_map else aggregateTimeSeries(images, exposure, country, dataset)

In [12]:
def agg_monthly_exposure_maps(exposure, country, dataset, facilities=False, draw_map=False):
    images=generateImageCollection(exposure, country, dataset, facilities)
    return cimatologyMaps(images, exposure, country, dataset)


# **Generate exposures and export**

In [11]:
lsat=monthly_exposures(exposure='lst', country='kenya', dataset='landsat', facilities=False)
lsat7=monthly_exposures(exposure='lst', country='kenya', dataset='landsat-7', facilities=False)
modis=monthly_exposures(exposure='lst', country='kenya', dataset='modis', facilities=False)
era5=monthly_exposures(exposure='lst', country='kenya', dataset='era5', facilities=False)
persiann=monthly_exposures(exposure='lst', country='kenya', dataset='persiann-cdr', facilities=False)
#merra2=monthly_exposures(exposure='lst', country='gambia', dataset='merra2', facilities=False)
trmm=monthly_exposures(exposure='lst', country='kenya', dataset='trmm', facilities=False)

In [13]:
lsat=lsat.rename(columns={'lst': 'landsat'})
lsat7=lsat7.rename(columns={'lst': 'landsat-7'})
modis=modis.rename(columns={'lst': 'modis'})
era5=era5.rename(columns={'lst': 'era5'})
trmm=trmm.rename(columns={'lst': 'trmm'})
persiann=persiann.rename(columns={'lst': 'persiann'})

lst=pd.merge(lsat, modis, left_index=True, right_index=True, how='outer', validate='1:1')
lst=pd.merge(lst, era5, left_index=True, right_index=True, how='outer', validate='1:1')
lst=pd.merge(lst, trmm, left_index=True, right_index=True, how='outer',suffixes=('_left','_right'), validate='1:1')
lst=pd.merge(lst, persiann, left_index=True, right_index=True, how='outer', validate='1:1')
lst=pd.merge(lst, lsat7, left_index=True, right_index=True, how='outer',suffixes=('_left1','_right1'), validate='1:1')[['landsat', 'modis', 'era5', 'trmm','persiann', 'landsat-7']]
lst

landsat      modis       era5      trmm  \
neighborhood_code exposure_month                                              
2540001           2018_11         35.135334  33.201024  28.238996  0.035018   
                  2018_12         39.326269  32.939137  28.951115  0.082926   
                  2019_01         40.472577  36.151111  29.773344  0.013784   
                  2019_02         36.547717  36.429708  30.494290  0.019060   
                  2019_03         35.714871  40.882270  31.635076  0.021344   
...                                     ...        ...        ...       ...   
2540051           2023_08         31.207844  31.184034  24.834646       NaN   
                  2023_09         42.482587  34.260225  26.052154       NaN   
                  2023_10         39.972696  36.236755  26.957408       NaN   
                  2023_11         35.957746  29.021185  25.617376       NaN   
                  2023_12         29.216726  30.086416  27.373188       NaN   

                                   persiann  landsat-7  
neighborhood_code exposure_month                        
2540001           2018_11          7.610682  33.841343  
                  2018_12          5.008244  31.747537  
                  2019_01          0.308883        NaN  
                  2019_02          0.796672  37.764665  
                  2019_03          0.741475  42.734808  
...                                     ...        ...  
2540051           2023_08          0.493064  23.850560  
                  2023_09          0.000000  27.342892  
                  2023_10          7.496070  28.018276  
                  2023_11         33.720305  17.748049  
                  2023_12          8.323994  30.574048  

[3162 rows x 6 columns]